In [871]:
import numpy as np
import pandas as pd
import re, nltk
from pprint import pprint

In [872]:

import types
import pandas as pd
from ibm_botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_d812e9ff73554acb99ec832832ddfe49 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-rPWCJ6tlad28m4TQMS_rhBCiHbQ4d2agItYcfqgqzen',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about your possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_d812e9ff73554acb99ec832832ddfe49.get_object(Bucket='progpy19cabd749c62e43b992119d84b67f744f', Key='AllRecipesInitial.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 


In [873]:
dataset_df = pd.read_excel(streaming_body_1)

In [874]:
dataset = dataset_df.copy()

In [875]:
#df_ingredient = dataset_df['ingredient']
#df_ingredient['ingredient'] = dataset_df['ingredient'].apply(lambda x: x.replace("_"," "))
#df_ingredient

In [876]:
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace(" ","_"))  #apply(lambda x: "%s" % '_'.join(x))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("agar_agaragar_agar", "agar_agar"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("yoghurt", "yogurt"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("e_cinnamon", "cinnamon"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("calorie_vanilla", "vanilla"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("milk_enough_water", "milk"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("vanilla_salt", "salt"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("vanilla_pod_split", "vanilla_pod"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("vanilla_bean_split", "vanilla_pod"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("vanilla_pod_split_seed", "vanilla_pod"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("vanilla_pod_seed", "vanilla_pod"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("c_fat_vanilla_wafer", "vanilla_wafer"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("flour_grain_salt", "salt"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("egg_salt", "egg"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("eggs", "egg"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("allspice_seed", "allspice"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("lemon_juice_concentrate", "lemon_juice"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("juice_lemon", "lemon_juice"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("cornstarch", "corn_starch"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("salt_filling", "salt"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("raspberry_butter", "raspberry"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("fig_date", "fig"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("nutmeg_water", "nutmeg"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("fat_cinnamon", "cinnamon"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("eden_agar", "agar_agar"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("fat_vanilla", "vanilla"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("buttermilk", "butter"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("light_muscovado_sugar", "muscovado_sugar"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("walnut_half", "walnut"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("nut_meat ","walnut")) 
#dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("white","white_Maltesers"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("grain_salt","salt"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("rosehip","rose_hip"))
#dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("semi_sweet","semisweet"))
#dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("semisweet","semi_sweet_baking_chocolate"))
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("gelatine","gelatin")) 
dataset['ingredient'] = dataset['ingredient'].apply(lambda x: x.replace("cardamon","cardamom")) 
dataset
    
data_df = dataset.drop(['recipe_title', 'amount', 'unit_of_measurement', 'instructions', 'url'], axis = 1)

# Dataset for testing result

In [877]:
import collections
data_collected = collections.Counter(data_df['ingredient'])
data_df_outputtest = {x : data_collected[x] for x in data_collected if data_collected[x] >= 30}


print("Length of Total Ingredient data: ", len(data_df['ingredient']))
print("Length of unique Ingredient data: ", len(data_collected))
print("Length of final output data: ", len(data_df_outputtest))
data_df_outputtest = list(data_df_outputtest.keys())
#data_df_outputtest[:10]

Length of Total Ingredient data:  198090
Length of unique Ingredient data:  4129
Length of final output data:  444


# Grouping the sentences by ID for creating model

In [878]:
sentences = data_df.groupby('ID')['ingredient'].apply(list)
#len(sentences)  # --> 22323

In [879]:
import gensim 
from gensim.models import Word2Vec
from ipykernel import kernelapp as app

In [910]:
# Create CBOW model
#model_2 = gensim.models.Word2Vec(sentences, min_count = 1, size = 250, window = 5, iter = 50, alpha = 0.03) 
model1 = gensim.models.Word2Vec(sentences, min_count =2, size = 250, window = 5, iter = 1000, alpha = 0.030) #300
# Create Skip Gram model 
#model2 = gensim.models.Word2Vec(sentences, min_count = 0, size = 100, window =5, sg = 2, iter = 100) 

# Similar words for words using CBOW

In [912]:
output_cbow = []
for i in range(len(data_df_outputtest)):
    input_test = data_df_outputtest[i]
    output_cbow.append((" {} ".format(input_test) + ":", model1.wv.similar_by_vector(input_test, topn=3)))
print("Similar words for words using CBOW")
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(output_cbow).head(100)

Similar words for words using CBOW


,0,1
0,mint_chocolate_chip :,"[(unsweetened_cocoa_powder, 0.295621782541275), (semi_sweet_chocolate, 0.29516762495040894), (chocolate_cooky, 0.2857244610786438)]"
1,white_bread :,"[(stale_white_bread, 0.36827605962753296), (grain_rice, 0.2937079668045044), (caramel_nut, 0.2863237261772156)]"
2,orange :,"[(orange_juice, 0.322617769241333), (curacao, 0.3039211332798004), (calvados_rum, 0.27033981680870056)]"
3,candied_orange_peel :,"[(sultana, 0.3081163763999939), (fig, 0.26166558265686035), (candied_orange, 0.25373148918151855)]"
4,juice :,"[(warm_cognac, 0.2540184557437897), (grape, 0.2385871410369873), (lump_sugar, 0.23092082142829895)]"
5,vanilla_pudding_mix :,"[(vanilla_pudding, 0.44542616605758667), (instant_vanilla_pudding, 0.34620335698127747), (instant_chocolate_pudding, 0.33940136432647705)]"
6,whisky :,"[(sherry_wine, 0.28694579005241394), (plantain, 0.26144200563430786), (snack_cake_mix, 0.25330090522766113)]"
7,oatmeal :,"[(oat, 0.5250821113586426), (rolled_oat, 0.49929744005203247), (quick_cooking_oat, 0.4132288098335266)]"
8,instant_coffee_powder :,"[(semisweet_chocolate, 0.3394230604171753), (unsweetened_chocolate, 0.32110852003097534), (chocolate, 0.2852395176887512)]"
9,maraschino_cherry :,"[(cherry, 0.2362319380044937), (coconut, 0.22894278168678284), (flaked_coconut, 0.224832683801651)]"


In [688]:
pd.DataFrame(output_cbow)[0]

0       mint_chocolate_chip :  
1       white_bread :          
2       orange :               
3       candied_orange_peel :  
4       juice :                
5       vanilla_pudding_mix :  
6       whisky :               
7       oatmeal :              
8       instant_coffee_powder :
9       maraschino_cherry :    
10      blueberry :            
11      graham_wafer :         
12      unbaked_pie_shell :    
13      quick_oat :            
14      chocolate_cake_mix :   
15      cranberry :            
16      apple_pie_filling :    
17      cornflour :            
18      cardamom :             
19      hot_water :            
20      dark_brown_sugar :     
21      lemon_cake_mix :       
22      softened_butter :      
23      walnut :               
24      white_chocolate :      
25      crisp_rice_cereal :    
26      whipped_topping :      
27      vanilla_pudding :      
28      crisco :               
29      yeast :                
         ...                   
414     

# Similar words for words using SkipGram

In [143]:
output_skipG = []
for i in range(15):
    input_test = data_df['ingredient'][i]
    output_skipG.append((" For {} ".format(input_test) + ":", model2.wv.similar_by_vector(input_test, topn=3)))
print("Similar words for words using SkipGram")
pd.DataFrame(output_skipG)

Similar words for words using SkipGram


,0,1
0,For gelatine :,"[(quark, 0.5243220329284668), (cream_liqueur, ..."
1,For baking_powder :,"[(baking_soda, 0.671516478061676), (salt, 0.63..."
2,For vanilla_pod :,"[(flan, 0.5421473979949951), (sweet_short_past..."
3,For banana :,"[(banana_liqueur, 0.4681349992752075), (vanill..."
4,For sugar :,"[(salt, 0.703315019607544), (granulated_sugar,..."
5,For salt :,"[(flour, 0.726675271987915), (sugar, 0.7033149..."
6,For muscovado_sugar :,"[(mixed_fruit_raisin_currant_sultana, 0.708096..."
7,For water :,"[(sugar, 0.5279949903488159), (prepared_graham..."
8,For double_cream :,"[(greek_yogurt, 0.6171751022338867), (caster_s..."
9,For sunflower_oil :,"[(loose_base, 0.5615432262420654), (beetroot, ..."


In [918]:
input_test = input("Enter the ingredient of which substitutes are required : ")

Enter the ingredient of which substitutes are required : clove


In [920]:
print("Similar words for {} -CBOW ".format(input_test) + "are:  ", model1.wv.similar_by_vector(input_test, topn=3))

Similar words for clove -CBOW are:   [('jaggery', 0.34004154801368713), ('whiskey_sauce', 0.32404106855392456), ('sweet_apple', 0.32219523191452026)]


In [85]:
print("Similar words for {} -SkipGram ".format(input_test) + "are:  ", model2.wv.similar_by_vector(input_test, topn=3))

Similar words for white_wine -SkipGram are:   [('plum', 0.9823551774024963), ('orange_marmalade', 0.9821877479553223), ('port', 0.9821869134902954)]


In [590]:
input_ing1 = input("write down the ingredient 1:  ")
input_ing2 = input("write down the ingredient 2:  ")
input_ing3 = input("write down the ingredient 3:  ")

write down the ingredient 3:  baking_powder


In [591]:
# Print results 
print("Cosine similarity between {} ".format(input_ing1) + "and {}- CBOW : ".format(input_ing2), model1.wv.similarity(input_ing1, input_ing2))
      
print("Cosine similarity between {} ".format(input_ing1) + "and {}- CBOW : ".format(input_ing3), model1.wv.similarity(input_ing1, input_ing3))

Cosine similarity between clove and nutmeg- CBOW :  0.311161
Cosine similarity between clove and baking_powder- CBOW :  0.194447


In [12]:
# Print results 
print("Cosine similarity between {} ".format(input_ing1) + "and {}- Skip Gram : ".format(input_ing2), model2.wv.similarity(input_ing1, input_ing2))      
print("Cosine similarity between {} ".format(input_ing1) + "and {}- Skip Gram : ".format(input_ing3), model2.wv.similarity(input_ing1, input_ing3))

Cosine similarity between vinegar and lemon_juice- Skip Gram :  0.628432
Cosine similarity between vinegar and liqueur- Skip Gram :  0.512905


In [399]:
#!pwd to get the tenant id
curl -O -X GET -u s8a6-af76c7a0464933-76e20627fd5b:41799493-7493-4323-a0c8-18d6a9437a1e -H 'X-Spark-service-instance-id:p-0e16a4f68d1452f5db09295ab29ff95c32ed4397, https://spark.bluemix.net/tenant/data/output_cbow.xlsx

SyntaxError: invalid syntax (<ipython-input-399-e87393d3b0b6>, line 2)

In [916]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(sc, '41799493-7493-4323-a0c8-18d6a9437a1e', 'p-0e16a4f68d1452f5db09295ab29ff95c32ed4397')
pc = project.project_context


In [378]:
!pwd      # this command provides tenant id.
!ls -l /gpfs/fs01/user/s8a6-af76c7a0464933-76e20627fd5b/notebook/work   # this command list down the files

/gpfs/fs01/user/s8a6-af76c7a0464933-76e20627fd5b/notebook/work
total 3341664
drwx------. 2 s8a6-af76c7a0464933-76e20627fd5b users      4096 Jul  2  2018 a2_m1.parquet
-rw-------. 1 s8a6-af76c7a0464933-76e20627fd5b users    209197 Jul  2  2018 a2_m1.parquet.zip
-rw-------. 1 s8a6-af76c7a0464933-76e20627fd5b users    282603 Jul  2  2018 a2_m1.parquet.zip.base64
drwx------. 2 s8a6-af76c7a0464933-76e20627fd5b users      4096 Jul 17  2018 a2_m2.json
-rw-------. 1 s8a6-af76c7a0464933-76e20627fd5b users     82333 Jul 17  2018 a2_m2.json.zip
-rw-------. 1 s8a6-af76c7a0464933-76e20627fd5b users    111225 Jul 17  2018 a2_m2.json.zip.base64
drwx------. 2 s8a6-af76c7a0464933-76e20627fd5b users      4096 Jul 16  2018 a2_m2.parquet
-rw-------. 1 s8a6-af76c7a0464933-76e20627fd5b users   1014954 Jul 16  2018 a2_m2.parquet.zip
-rw-------. 1 s8a6-af76c7a0464933-76e20627fd5b users   1371079 Jul 16  2018 a2_m2.parquet.zip.base64
drwx------. 2 s8a6-af76c7a0464933-76e20627fd5b users      4096 Jul 10  2018 a

In [917]:
# let's assume you have the pandas DataFrame  pandas_df which contains the data
# you want to save in your object storage as a csv file
project.save_data("output_cbow_3.csv", (pd.DataFrame(output_cbow)).to_csv(), overwrite = True)

# Get the url
url = project.get_file_url("output_cbow_3.csv")

# Fetch the CSV file from the object storage using Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(url)
df_data_1.show(5)


+---+--------------------+--------------------+
|_c0|                   0|                   1|
+---+--------------------+--------------------+
|  0| mint_chocolate_c...|[('unsweetened_co...|
|  1|       white_bread :|[('stale_white_br...|
|  2|            orange :|[('orange_juice',...|
|  3| candied_orange_p...|[('sultana', 0.30...|
|  4|             juice :|[('warm_cognac', ...|
+---+--------------------+--------------------+
only showing top 5 rows



In [398]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Download CSV file", filename = "output_cbow.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(pd.DataFrame(output_cbow))